In [3]:
from datasets import load_from_disk
tokenized_dataset = load_from_disk('./model3-outputs-gpt2tokenizer-varlength')
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
def tokenize(element):
    ctx_length = 512
    input_batch = []
    count  = 0
    for input_ids in element['input_ids']:
        while len(input_ids) < ctx_length:
            #print('im here! len is ', len(input_ids), ' count is ', count)
            input_ids.append(tokenizer.eos_token_id)
        input_batch.append(input_ids)
        count += 1
    return {'input_ids': input_batch}
import numpy as np
from datasets import DatasetDict
my_batch = DatasetDict({
    'train': tokenized_dataset['train'].select(np.arange(50)),
    'validation': tokenized_dataset['validation'].select(np.arange(50))
})
my_batch
my_batch = my_batch.map(tokenize, batched=True, remove_columns=my_batch['train'].column_names)

/home/t-kgatmiry/miniconda3/envs/env3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
my_tensor = torch.tensor(my_batch['train']['input_ids'])
flat_tensor = my_tensor.flatten()
my_tensor.shape

flat_index = (flat_tensor == 50256).nonzero(as_tuple=False)

In [ ]:
torch.set_printoptions(profile="full")
print(((my_tensor == 50256).cumsum(1) == 1).argsort())

In [7]:
myten = torch.tensor([[1,0],[2,4]])
myten.cumsum(1)

tensor([[1, 1],
        [2, 6]])

In [3]:
import torch

batch_size = 2
seq_len = 5
num_classes = 10

# Logits output from the model (before softmax)
logits = torch.randn(batch_size, seq_len, num_classes)

# Ground truth labels (with padding token -1)
labels = torch.tensor([
    [1, 2, -1, 4, 5],
    [4, -1, 6, 7, 8]
])

mask = (labels != -1)

In [14]:
logits_flat = logits.view(-1, num_classes)
logits_flat
labels_flat = labels.view(-1)
masked_loss = (labels != -1)
masked_loss_flat = masked_loss.view(-1)
logits_flat.shape
#logits_flat[masked_loss_flat].shape

torch.Size([10, 10])

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
loss = loss_fn(logits_flat, labels_flat)

In [7]:
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.decode(torch.tensor([13]))

'.'

In [3]:
from datasets import load_dataset
load_dataset('tinystories-withsummary-gpt2tokenizer')

/home/t-kgatmiry/miniconda3/envs/env3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 2200000
    })
})

In [7]:
combined_dataset_dict = load_dataset('json', data_files='/home/t-kgatmiry/sandbox2225-T4/tinystories/data00.json')

In [6]:
import torch
from datasets import concatenate_datasets
from datasets import DatasetDict
from datasets import load_dataset
#with open('/home/t-kgatmiry/sandbox2225-T4/tinystories/data01.json', 'r') as file:
#    obj = json.load(file)


for i in range(8,10):
    mydict = load_dataset('json', data_files='/home/t-kgatmiry/sandbox2225-T4/tinystories/data0'+ str(i) +'.json')
    for split in mydict.keys():
        combined_dataset_dict[split] = concatenate_datasets([mydict[split], combined_dataset_dict[split]])


In [5]:
mydict = load_dataset('json', data_files='/home/t-kgatmiry/sandbox2225-T4/tinystories/data01.json')
for split in mydict.keys():
    combined_dataset_dict[split] = concatenate_datasets([mydict[split], combined_dataset_dict[split]])

NameError: name 'concatenate_datasets' is not defined

In [95]:
combined_dataset_dict



DatasetDict({
    train: Dataset({
        features: ['instruction', 'story', 'summary', 'source'],
        num_rows: 2100000
    })
})

In [32]:
from datasets import load_dataset
ds = load_dataset("roneneldan/TinyStories")
ds

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2119719
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 21990
    })
})

In [1]:
def tokenize(element):
   outputs = tokenizer(element['story'])['input_ids']
   outputs2 = tokenizer(element['summary'])['input_ids']
   story_input_batch = []
   summary_input_batch = []
   for input_ids in outputs:
      input_ids.append(tokenizer.eos_token_id)
      story_input_batch.append(input_ids)
   for input_ids in outputs2:
      input_ids.append(tokenizer.eos_token_id)
      summary_input_batch.append(input_ids) 
   return {'story_input_ids': story_input_batch, 'summary_input_ids': summary_input_batch}

In [9]:
def tokenize(element):
   outputs = tokenizer(element['story'])['input_ids']
   outputs2 = tokenizer(element['summary'])['input_ids']
   input_batch = []
   for i in range(len(outputs)):
      outputs[i].append(tokenizer.eos_token_id)
      outputs2[i].append(tokenizer.eos_token_id)
      input_batch.append(outputs[i] + [-1] + outputs2[i])
   return {'input_ids': input_batch}

In [97]:
tokenized_dataset = combined_dataset_dict.map(tokenize, batched=True, remove_columns=combined_dataset_dict['train'].column_names)

Map: 100%|██████████| 2100000/2100000 [03:27<00:00, 10132.31 examples/s]


In [2]:
tokenized_dataset.save_to_disk('./tinystories-withsummary-gpt2tokenizer')

NameError: name 'tokenized_dataset' is not defined

In [1]:
#len(tokenized_dataset['train']['input_ids'][1000])
tokenized_dataset = load_dataset('./tinystories-withsummary-gpt2tokenizer')

NameError: name 'tokenized_dataset' is not defined

In [22]:
vocab = tokenizer.get_vocab()
min_token_id = max(vocab.values())
min_token_id

50256

In [25]:
import torch
myten = torch.tensor([1,2,3])
mylist = myten.tolist()
mylist

[1, 2, 3]

In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')


def compute(x):
    # Example I/O-bound task (e.g., fetching a URL)
    tokenized_story = tokenizer(x['story'])['input_ids']
    tokenized_story.append(tokenizer.eos_token_id)
    tokenized_summary = tokenizer(x['summary'])['input_ids']
    tokenized_summary.append(tokenizer.eos_token_id)
    return (tokenized_story, tokenized_summary)

def parallel_compute(data):
    results = []
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(compute, item) for item in data]
        for future in as_completed(futures):
            results.append(future.result())
    return results

In [16]:
results = parallel_compute(obj[1:20])
len(results)

19

In [4]:
from transformers import DataCollatorForLanguageModeling, AutoTokenizer, GPT2Model, AutoModel
tokenizer = AutoTokenizer.from_pretrained('gpt2')
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
tokenizer.pad_token = tokenizer.eos_token
mytensor = data_collator([[1,2,3], [4,5]])
mytensor

{'input_ids': tensor([[    1,     2,     3],
         [    4,     5, 50256]]),
 'labels': tensor([[   1,    2,    3],
         [   4,    5, -100]])}

In [4]:
myten = torch.tensor([[-1,1,2],[4,-1,6]])
myten[0, :1]

tensor([-1])

In [1]:
from mymodelsummary import MyModel
from transformers import AutoConfig
import torch
config = AutoConfig.from_pretrained('gpt2')
mymodel = MyModel(config)
mymodel.to('cuda')

input_ids = torch.tensor([[1,7,-1,2], [4,-1,6,9]]).to('cuda')
mymodel(input_ids)
#separation_indices = (input_ids == -1).nonzero()[:,1]


/home/t-kgatmiry/miniconda3/envs/env3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


input_ids_main is  tensor([[    1,     7, 50256],
        [    4, 50256, 50256]], device='cuda:0')  input_ids_summary is  tensor([[    2, 50256, 50256],
        [    6,     9, 50256]], device='cuda:0')


{'loss': tensor(10.8475, device='cuda:0', grad_fn=<NllLossBackward0>),
 'logits': tensor([[[ 0.5728,  0.8075, -0.0230,  ...,  0.1650, -0.2879, -0.0997],
          [ 0.7980,  0.9170,  0.4490,  ...,  0.2185,  0.8730, -0.0370],
          [-0.5922,  0.5058, -0.5181,  ..., -0.3854,  0.0182, -0.2702]],
 
         [[ 0.6189,  0.5544,  0.4526,  ...,  0.6348, -0.1919, -0.3053],
          [ 0.3903,  0.9254,  0.2195,  ...,  0.1465,  0.8714,  0.4362],
          [-0.9304,  0.7725, -0.5905,  ...,  0.2377,  0.4106,  0.2702]]],
        device='cuda:0', grad_fn=<SliceBackward0>)}

In [2]:
input_ids_summary = torch.tensor([[1,-1,-1],[4,3,-1]])
((input_ids_summary == -1).cumsum(1) == 1)

tensor([[False,  True, False],
        [False, False,  True]])

In [7]:
[input_ids[i , :separation_indices[i]].tolist().append(-100) for i in range(input_ids.shape[0])]

[None, None]

In [3]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk('./tinystories-withsummary-gpt2tokenizer-49')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 4867871
    })
})